# Kapitola 7: Tvorba chatovacích aplikácií
## Rýchly úvod do OpenAI API

Tento notebook je upravený z [repozitára Azure OpenAI Samples](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), ktorý obsahuje notebooky využívajúce služby [Azure OpenAI](notebook-azure-openai.ipynb).

Python OpenAI API funguje aj s modelmi Azure OpenAI, s niekoľkými úpravami. Viac o rozdieloch sa dozviete tu: [Ako prepínať medzi OpenAI a Azure OpenAI endpointmi v Pythone](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# Prehľad  
„Veľké jazykové modely sú funkcie, ktoré mapujú text na text. Pri zadaní vstupného reťazca textu sa veľký jazykový model snaží predpovedať, aký text bude nasledovať“(1). Tento „rýchly štart“ notebook predstaví používateľom základné pojmy o LLM, kľúčové požiadavky balíkov na začatie práce s AML, jednoduchý úvod do tvorby promptov a niekoľko krátkych príkladov rôznych spôsobov využitia.


## Obsah  

[Prehľad](../../../../07-building-chat-applications/python)  
[Ako používať OpenAI Service](../../../../07-building-chat-applications/python)  
[1. Vytvorenie vášho OpenAI Service](../../../../07-building-chat-applications/python)  
[2. Inštalácia](../../../../07-building-chat-applications/python)    
[3. Prihlasovacie údaje](../../../../07-building-chat-applications/python)  

[Príklady použitia](../../../../07-building-chat-applications/python)    
[1. Zhrnutie textu](../../../../07-building-chat-applications/python)  
[2. Klasifikácia textu](../../../../07-building-chat-applications/python)  
[3. Generovanie nových názvov produktov](../../../../07-building-chat-applications/python)  
[4. Doladenie klasifikátora](../../../../07-building-chat-applications/python)  

[Referencie](../../../../07-building-chat-applications/python)


### Vytvorte svoj prvý prompt  
Toto krátke cvičenie vám poskytne základný úvod do zadávania promptov modelu OpenAI pre jednoduchú úlohu „zhrnutie“.


**Kroky**:  
1. Nainštalujte knižnicu OpenAI do svojho python prostredia  
2. Načítajte štandardné pomocné knižnice a nastavte svoje bežné bezpečnostné údaje pre službu OpenAI, ktorú ste vytvorili  
3. Vyberte model pre svoju úlohu  
4. Vytvorte jednoduchý prompt pre model  
5. Odošlite svoju požiadavku do modelového API!


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. Výber správneho modelu  
Modely GPT-3.5-turbo alebo GPT-4 dokážu rozumieť a generovať prirodzený jazyk.


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. Návrh promptov  

„Kúzlo veľkých jazykových modelov spočíva v tom, že tým, že sa učia minimalizovať chybu predikcie na obrovskom množstve textu, modely sa nakoniec naučia koncepty užitočné pre tieto predikcie. Napríklad sa naučia koncepty ako“(1):

* ako sa správne píše
* ako funguje gramatika
* ako parafrázovať
* ako odpovedať na otázky
* ako viesť konverzáciu
* ako písať v rôznych jazykoch
* ako programovať
* atď.

#### Ako ovládať veľký jazykový model  
„Zo všetkých vstupov do veľkého jazykového modelu je jednoznačne najvplyvnejší textový prompt.“(1)

Veľké jazykové modely môžu byť navádzané na generovanie výstupu niekoľkými spôsobmi:

Inštrukcia: Povedzte modelu, čo chcete
Doplnenie: Navádzajte model, aby doplnil začiatok toho, čo chcete
Ukážka: Ukážte modelu, čo chcete, buď:
Niekoľko príkladov priamo v promte
Stovky alebo tisíce príkladov v trénovacej dátovej sade na doladenie



#### Existujú tri základné odporúčania pre tvorbu promptov:

**Ukážte a vysvetlite.** Jasne dajte najavo, čo chcete, buď prostredníctvom inštrukcií, príkladov alebo ich kombinácie. Ak chcete, aby model zoradil zoznam položiek podľa abecedy alebo aby klasifikoval odsek podľa sentimentu, ukážte mu, že toto je vaším cieľom.

**Používajte kvalitné dáta.** Ak sa snažíte vytvoriť klasifikátor alebo chcete, aby model nasledoval určitý vzor, uistite sa, že máte dostatok príkladov. Dôkladne si skontrolujte svoje príklady — model je zvyčajne dosť inteligentný na to, aby prehliadol základné pravopisné chyby a odpovedal vám, ale môže si tiež myslieť, že to bolo zámerné, čo môže ovplyvniť odpoveď.

**Skontrolujte svoje nastavenia.** Nastavenia temperature a top_p ovplyvňujú, nakoľko je model deterministický pri generovaní odpovede. Ak od neho očakávate odpoveď, kde je len jedna správna možnosť, nastavte tieto hodnoty nižšie. Ak chcete rozmanitejšie odpovede, nastavte ich vyššie. Najčastejšou chybou pri týchto nastaveniach je predpoklad, že ide o ovládanie „šikovnosti“ alebo „kreativity“ modelu.


Zdroj: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Zhrnutie textu  
#### Úloha  
Zhrňte text pridaním 'tl;dr:' na koniec textového úseku. Všimnite si, ako model dokáže vykonávať množstvo úloh bez ďalších inštrukcií. Môžete experimentovať s opisnejšími promptmi ako tl;dr, aby ste upravili správanie modelu a prispôsobili si výsledné zhrnutie(3).  

Nedávne výskumy ukázali výrazné zlepšenia v mnohých NLP úlohách a benchmarkoch vďaka predtrénovaniu na veľkom korpuse textov, po ktorom nasleduje doladenie na konkrétnu úlohu. Hoci je táto metóda zvyčajne architektonicky nezávislá od úlohy, stále vyžaduje špecifické datasety na doladenie s tisíckami alebo desaťtisícami príkladov. Naopak, ľudia dokážu zvládnuť novú jazykovú úlohu už po niekoľkých príkladoch alebo jednoduchých inštrukciách – čo je niečo, s čím majú súčasné NLP systémy stále problém. Tu ukazujeme, že zväčšenie jazykových modelov výrazne zlepšuje výkonnosť v úlohách s malým počtom príkladov bez ohľadu na úlohu, niekedy dokonca dosahuje úroveň konkurencie s predchádzajúcimi najlepšími prístupmi založenými na doladení. 



Tl;dr


# Cvičenia pre rôzne použitia  
1. Zhrnúť text  
2. Klasifikovať text  
3. Vytvoriť nové názvy produktov


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Klasifikácia textu  
#### Výzva  
Zaradiť položky do kategórií, ktoré sú zadané počas inferencie. V nasledujúcom príklade zadávame v promptu (*playground_reference) kategórie aj text, ktorý treba klasifikovať.

Dopyt zákazníka: Dobrý deň, na mojej klávesnici notebooku sa mi nedávno zlomil jeden kláves a budem potrebovať náhradný:

Zaradená kategória:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Generovanie nových názvov produktov
#### Výzva
Vytvorte názvy produktov na základe príkladových slov. V tomto zadaní uvádzame informácie o produkte, pre ktorý ideme generovať názvy. Tiež poskytujeme podobný príklad, aby sme ukázali vzor, aký chceme dostať. Nastavili sme aj vysokú hodnotu parametra temperature, aby boli odpovede náhodnejšie a inovatívnejšie.

Popis produktu: Domáci mixér na milkshake
Kľúčové slová: rýchly, zdravý, kompaktný.
Názvy produktov: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Popis produktu: Pár topánok, ktoré sa prispôsobia akejkoľvek veľkosti nohy.
Kľúčové slová: prispôsobivý, fit, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Referencie  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Príklady](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Najlepšie postupy pre doladenie GPT-3 na klasifikáciu textu](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Pre viac pomoci  
[OpenAI Komerčný tím](AzureOpenAITeam@microsoft.com)


# Prispievatelia
* Louis Li



---

**Vyhlásenie o vylúčení zodpovednosti**:  
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Za autoritatívny zdroj by sa mal považovať pôvodný dokument v jeho natívnom jazyku. Pre kritické informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
